In [40]:
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from detect_outlier import *
from sklearn.model_selection import train_test_split


In [41]:
df = read_data()
df = df.dropna()

In [42]:
df_normalized =df.drop("address", axis= 1).drop("createdAt", axis= 1)
label_column = df['main_label']
sub_label_column = df['sub_label']
df_normalized = (df_normalized - df_normalized.min()) / (df_normalized.max() - df_normalized.min())
df_normalized = df_normalized * 550 + 300
df_normalized['main_label'] = main_label_column
df_normalized['sub_label'] = sub_label_column


In [43]:
def score(label):
    if label == 0:
        return np.random.randint(300, 580)
    if label == 1:
        return np.random.randint(580, 670)
    if label == 2:
        return np.random.randint(670, 740)
    if label == 3:
        return np.random.randint(740, 800)
    if label == 4:
        return np.random.randint(800, 850)


def labeling(scores):
    label = []
    for score in scores:
        if score < 580:
            label.append(0)
        elif score >= 580 and score < 670:
            label.append(1)
        elif score >= 670 and score < 740:
            label.append(2)
        elif score >= 740 and score < 800:
            label.append(3)
        elif score >= 800:
            label.append(4)
    return np.array(label)

In [44]:
X = np.array(df_normalized[
    [
        "balanceInUSD",
        # "borrowInUSD",
        # "depositInUSD",
        "averageTotalAsset",
        "frequencyMountOfTransaction",
        "borrow_per_balance",
        "deposit_per_balance",
        "borrow_per_deposit",
        "totalValueOfLiquidation",
        "numberOfLiquidation",
        "frequencyOfTransaction",
        "frequencyOfDappTransactions",
        "numberOfInteractedDapps",
        "typesOfInteractedDapps",
        "numberOfReputableDapps",
        "age",
    ]
].values)
main_y = df_normalized["main_label"].values
sub_y = df_normalized["sub_label"].values

X_train, X_test, main_y_train, main_y_test, sub_y_train, sub_y_test  = train_test_split(
            X, main_y, sub_y, test_size=0.2, random_state=42
        )
# Chuẩn bị dữ liệu


In [45]:
X_train.shape[0]

16729

In [46]:
import numpy as np

# Hàm tính giá trị của hàm lỗi (MSE)
def mean_squared_error(X, y, sub_y,  w):
    n = len(y)
    y_pred = labeling(np.dot(X, w))
    mse = np.sum((y - y_pred) ** 2) / n
    return mse

# Hàm tính gradient của hàm lỗi (MSE)
def gradient_mean_squared_error(X, y, w):
    n = len(y)
    y_pred = labeling(np.dot(X, w))
    error = y_pred - y
    # TODO: Modify Error Function
    gradient = 2 * np.dot(X.T, error) / n
    return gradient

# Gradient descent để tối ưu hóa hàm lỗi (MSE)
def adam(X, y, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8, num_iterations=1000):
    # Khởi tạo vector trọng số ngẫu nhiên
    w = np.random.rand(X.shape[1])
    m = np.zeros_like(w)
    v = np.zeros_like(w)
    t = 0
    
    for _ in range(num_iterations):
        # Chọn ngẫu nhiên một mẫu từ dữ liệu
        random_index = np.random.randint(0, len(X))
        X_sample = X[random_index:random_index+1]
        y_sample = y[random_index:random_index+1]
        
        # Tính gradient của hàm lỗi cho mẫu đã chọn
        grad = gradient_mean_squared_error(X_sample, y_sample, w)
        
        # Tính trung bình độ lớn của gradient và gradient bình phương
        m = beta1 * m + (1 - beta1) * grad
        v = beta2 * v + (1 - beta2) * (grad ** 2)
        
        # Bias correction
        m_hat = m / (1 - beta1**(t+1))
        v_hat = v / (1 - beta2**(t+1))
        
        # Cập nhật vector trọng số
        w -= learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)
        
        t += 1
        
    
    return w

# Áp dụng gradient descent để tối ưu hóa hàm lỗi (MSE)
learned_weights = adam(X_train, main_y_train)

print("Vector trọng số tối ưu:", learned_weights)


Vector trọng số tối ưu: [ 0.44882406 -0.08729184 -0.10223234 -0.28768874 -0.15245593 -0.19518524
  0.27720341  0.2622802  -0.07231391  0.17860991  0.57653186  0.60278579
  0.51849399  0.27414808]


In [47]:
pred = labeling(X_test.dot(learned_weights))
print(accuracy_score(y_test, pred))

0.7002151565861822


In [48]:
import numpy as np

unique_elements, counts = np.unique(pred, return_counts=True)
    
for element, count in zip(unique_elements, counts):
    print("Phần tử", element, "xuất hiện", count, "lần")


Phần tử 0 xuất hiện 1 lần
Phần tử 1 xuất hiện 1 lần
Phần tử 2 xuất hiện 3698 lần
Phần tử 3 xuất hiện 457 lần
Phần tử 4 xuất hiện 26 lần


In [49]:
# Gradient descent với AdaMax để tối ưu hóa hàm lỗi (MSE)
def adamax_gradient_descent(X, y, learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8, num_iterations=1000):
    # Khởi tạo vector trọng số ngẫu nhiên
    w = np.random.randn(X.shape[1])
    
    # Khởi tạo các biến m và v cho Adam
    m = np.zeros_like(w)
    v = np.zeros_like(w)
    
    for t in range(1, num_iterations + 1):
        # Tính gradient của hàm lỗi
        grad = gradient_mean_squared_error(X, y, w)
        
        # Cập nhật biến m và v
        m = beta1 * m + (1 - beta1) * grad
        v = np.maximum(beta2 * v, np.abs(grad))
        
        # Cập nhật vector trọng số
        w -= learning_rate / (1 - beta1**t) * m / (v + epsilon)
        if _ % 100 == 0:
            print(w)
    
    return w

In [50]:
learned_weights = adamax_gradient_descent(X_train, y_train)
print("Vector trọng số tối ưu:", learned_weights)
pred = labeling(X_test.dot(learned_weights))
print(accuracy_score(y_test, pred))

Vector trọng số tối ưu: [-0.59874422 -0.72478254  0.62521165 -0.52890806  2.15827971  0.38619855
 -1.06429626 -0.05813661 -0.13724531 -0.10041041  0.45887657  0.39231388
 -0.12534711  1.12314901]
0.545063351661487


In [51]:
import numpy as np

unique_elements, counts = np.unique(pred, return_counts=True)

for element, count in zip(unique_elements, counts):
    print("Phần tử", element, "xuất hiện", count, "lần")


Phần tử 0 xuất hiện 90 lần
Phần tử 1 xuất hiện 509 lần
Phần tử 2 xuất hiện 2749 lần
Phần tử 3 xuất hiện 561 lần
Phần tử 4 xuất hiện 274 lần
